In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

# Preliminary functions
Functions that make our life easier, since we will be reusing some code.

In [2]:
def dimensions(molecule):
    """
    Return the dimensions (N, M) of a molecule in a given basis. Here, N is the number of electrons and M the number of spinors.
    """
    N = molecule.numberOfElectrons()

    spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
    M = spin_orbital_basis.numberOfSpinors()
    
    return (N, M)

In [3]:
def onv_bases(N, K, transition = "detachment"):
    """
    Create two ONV bases, based on whether we are describing an electron attachment or detachment.
    """

    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2
    
    if transition == "detachment":
        onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta - 1)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        
    elif transition == "attachment":
        onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_alpha + 1, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
        
    else:
        print("Specify a valid electron transition process: attachment/detachment.")
        return None
    
    return (onv_basis_I, onv_basis_J)

In [4]:
def C_RHF(molecule):
    
    N = molecule.numberOfElectrons()
    N_P = molecule.numberOfElectronPairs()
    
    spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spin_orbital_basis.numberOfSpatialOrbitals()
    #print("N: {}\nN_P: {}\nK: {}".format(N, N_P, K))

    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)
    S = spin_orbital_basis.quantizeOverlapOperator().parameters()

    environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S)
    solver = gqcpy.RHFSCFSolver.Plain()
    objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # use the default threshold of 1.0e-08
    rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
    
    return rhf_parameters.expansion()

In [5]:
def C_UHF(molecule):
    N = molecule.numberOfElectrons()
    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2

    spinor_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spinor_basis.numberOfSpatialOrbitals()
    #print("N_alpha: {}\nN_beta: {}\nK: {}".format(N_alpha, N_beta, K))
    
    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spinor_basis, molecule) 
    S = spinor_basis.quantizeOverlapOperator().parameters()
    
    environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, hamiltonian, S)
    solver = gqcpy.UHFSCFSolver.Plain()
    uhf_parameters = gqcpy.UHF.optimize(solver, environment).groundStateParameters()
    return uhf_parameters.expansion()

In [6]:
def TM(C_ion, C_neutral):
    return C_ion.inverse().transformed(C_neutral)

# Preparation of some variables
Generate the neutral molecule with N electrons, a positive ion with N-1 electrons (charge +1) and a negative ion with N+1 electrons (charge -1).

In [7]:
#neutral_molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2.xyz" , 0)  # create a neutral molecule with N electrons
#positive_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2.xyz" , +1)  # create a an ion N-1 electrons
#negative_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2.xyz" , -1)  # create a an ion with N+1 electrons

neutral_molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , 0)  # create a neutral molecule with N electrons
positive_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion N-1 electrons
negative_ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , -1)  # create a an ion with N+1 electrons

print("Neutral molecule:")
print(neutral_molecule)
print("\nPositive ion:")
print(positive_ion)
print("\nNegative ion:")
print(negative_ion)

Neutral molecule:
Number of electrons: 10 
O  (0, -0.143226, 0)
H  (1.63804, 1.13655, -0)
H  (-1.63804, 1.13655, -0)


Positive ion:
Number of electrons: 9 
O  (0, -0.143226, 0)
H  (1.63804, 1.13655, -0)
H  (-1.63804, 1.13655, -0)


Negative ion:
Number of electrons: 11 
O  (0, -0.143226, 0)
H  (1.63804, 1.13655, -0)
H  (-1.63804, 1.13655, -0)



In [8]:
N, M = dimensions(neutral_molecule)
K = M//2 # number of spinors per spin compound

onv_basis_I, onv_basis_J = onv_bases(N, K, "detachment")

# Fukui matrix in the uncorrelated limit

In [9]:
C_neutral = C_UHF(neutral_molecule)
C_ion = C_UHF(positive_ion)

In [10]:
U = TM(C_ion, C_neutral)

In [11]:
print("Is C_ion unitary? ", C_ion.isUnitary())
print("Is C_neutral unitary? ", C_neutral.isUnitary())
print("Is U unitary? ", U.isUnitary())

Is C_ion unitary?  False
Is C_neutral unitary?  False
Is U unitary?  True


In [12]:
Psi_I = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_I)
#Psi_I.coefficients()
Psi_J = gqcpy.LinearExpansion_SpinResolved.HartreeFock(onv_basis_J)
#Psi_J.coefficients()

In [13]:
DM_I = Psi_I.calculateSpinResolved1DM()
DM_J = Psi_J.calculateSpinResolved1DM()

In [14]:
DM_I = DM_I.transformed(U)

fukui_matrix_neg = (DM_J - DM_I).orbitalDensity()
fukui_eigenvalues, fukui_naturals = np.linalg.eigh(fukui_matrix_neg)
np.round(fukui_naturals,3)

array([[-0.001, -0.   ,  1.   , -0.023, -0.   , -0.002,  0.   ],
       [-0.23 , -0.   ,  0.021,  0.933, -0.   , -0.275,  0.   ],
       [ 0.   , -0.653,  0.   , -0.   , -0.758,  0.   ,  0.   ],
       [ 0.607,  0.   ,  0.01 ,  0.358,  0.   ,  0.709,  0.   ],
       [ 0.   , -0.   , -0.   , -0.   , -0.   ,  0.   , -1.   ],
       [ 0.761,  0.   ,  0.   , -0.004, -0.   , -0.649,  0.   ],
       [-0.   ,  0.758,  0.   ,  0.   , -0.653,  0.   ,  0.   ]])

In [15]:
fukui_eigenvalues

array([-2.98678643e-01, -2.74420688e-01, -6.65719147e-16,  7.44488336e-04,
        2.74420688e-01,  2.97934154e-01,  1.00000000e+00])